In [1]:
# Basic Python/Data Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import re


# Add this import at the top of your cell
from IPython.display import display


# Salesforce Integration
from simple_salesforce import Salesforce
import requests

# Forecasting Library
from prophet import Prophet
# from prophet.plot import plot_plotly, plot_components_plotly # For interactive plots

# Pandas Display Settings
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', 1000) 

# Matplotlib/Seaborn Style
plt.style.use('seaborn-v0_8-whitegrid')


pd.set_option('display.width', 2000)      # Set a large width to prevent line wrapping
pd.set_option('display.max_columns', None)  # Ensure all columns are shown
pd.set_option('display.max_rows', 200)      # Optional: show more rows if needed
pd.set_option('display.float_format', '{:,.2f}'.format) # Optional: format numbers for readability


print("Cell 1: Imports and Global Settings - Executed")

Cell 1: Imports and Global Settings - Executed


In [2]:
# --- General Percentage Change Thresholds ---
# A metric will be flagged as a "Drop/Decrease" if its value falls by this percentage or more. Use a negative number.
PCT_CHANGE_THRESHOLD_DROP = -20

# A metric will be flagged as an "Increase" if its value rises by this percentage or more. Use a positive number.
PCT_CHANGE_THRESHOLD_INCREASE = 20



# --- Absolute Change Thresholds (used in combination with percentage changes for some metrics) ---
# For 'Total ARR', a change must meet BOTH the percentage threshold AND this absolute dollar amount to be flagged.
# This prevents flagging a 50% drop on a very small number (e.g., from $100 to $50) as "significant".
ABS_ARR_CHANGE_THRESHOLD = 50000

# For '# of Opps' and '# of Won Opps', a change must meet BOTH the percentage threshold AND this absolute opportunity count.
# This avoids flagging a 100% change from 1 opp to 2 opps as significant.
ABS_OPP_CHANGE_THRESHOLD = 5
# For 'Avg Sales Price', a change must meet BOTH the percentage threshold AND this absolute dollar amount.
ASP_CHANGE_THRESHOLD = 500


# --- Specific Metric Thresholds (these work independently) ---
# For 'Win Rate (Count)', flags a change if the rate moves by this many absolute percentage points.
# Example: If set to 5, a change from 30% to 24% (a -6 point change) would be flagged as a significant drop.
# This is often more useful than a relative percentage change for win rates.
WIN_RATE_PCT_POINT_CHANGE_THRESHOLD = 5

# For 'Avg Sales Cycle', a change is flagged if it meets EITHER the absolute day change OR the percentage change threshold.
# This captures both small-but-fast relative changes and large absolute changes.
# Flags an increase/decrease if the sales cycle changes by at least this many days.
SALES_CYCLE_DAY_CHANGE_THRESHOLD = 10
# Flags an increase/decrease if the sales cycle changes by at least this percentage.
SALES_CYCLE_PCT_CHANGE_THRESHOLD = 15

# --- Stage Name Configuration ---
# Define the exact names of your "Won" and "Lost" stages from Salesforce.
# This is critical for calculating Win Rate correctly.
STAGE_WON = 'Closed Won'
STAGE_LOST = 'Closed Lost'

# ## Salesforce Configuration & Credentials

# --- USER: Salesforce Credentials & Report ID ---
SF_USERNAME = 'bchen@envoy.com' # Replace with your username
SF_PASSWORD = 'TasksandEvents1'   # Replace with your password
SF_SECURITY_TOKEN = 'nQWlT8vNdnJwxwtfpS1ic4Z7O' # Replace with your security token
SF_INSTANCE_URL = 'https://envoy.my.salesforce.com/'
MAIN_REPORT_ID = '00OUO000009IZVD2A4' # Replace with your main Salesforce Report ID

# --- Other Configurations ---
# Fiscal year definition: FY2023 starts on Feb 1, 2022. We will filter for data from FY2023 onwards.
START_FISCAL_YEAR_FOR_ANALYSIS = 2023 

print(f"Cell 2: Salesforce Configuration - User: {SF_USERNAME}, Report: {MAIN_REPORT_ID}, Start FY: {START_FISCAL_YEAR_FOR_ANALYSIS} - Set")

Cell 2: Salesforce Configuration - User: bchen@envoy.com, Report: 00OUO000009IZVD2A4, Start FY: 2023 - Set


In [3]:
# --- Function to Download Salesforce Report ---
def download_sf_report(sf_instance, report_id, sf_session):
    if sf_session is None:
        print("Salesforce session not available. Cannot download report.")
        return pd.DataFrame()
    export_params = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
    sf_url = f"{sf_instance}{report_id}{export_params}"
    try:
        response = requests.get(sf_url, headers=sf_session.headers, cookies={'sid': sf_session.session_id})
        response.raise_for_status()
        report_content = response.content.decode('utf-8')
        lines = report_content.splitlines()
        num_footer_lines = 0
        for i in range(len(lines) - 1, max(0, len(lines) - 10), -1): # Check last 10 lines
            line_strip = lines[i].strip()
            if "Confidential Information - Do Not Distribute" in line_strip or \
               "Report Generation Date" in line_strip or \
               "Envoy" == line_strip or \
               len(line_strip) == 0:
                num_footer_lines +=1
            else:
                break
        report_content_cleaned = "\n".join(lines[:-num_footer_lines]) if num_footer_lines > 0 else report_content
        df_report = pd.read_csv(StringIO(report_content_cleaned))
        print(f"Report {report_id} downloaded successfully. Shape: {df_report.shape}")
        return df_report
    except requests.exceptions.RequestException as e:
        print(f"Error downloading report {report_id}: {e}")
        if 'response' in locals() and response is not None: print(f"Response content (first 500 chars): {response.text[:500]}")
        return pd.DataFrame()
    except pd.errors.EmptyDataError:
        print(f"Report {report_id} is empty or not parsed correctly.")
        if 'report_content' in locals(): print(f"Raw content (first 500 chars): {report_content[:500]}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred while processing report {report_id}: {e}")
        if 'report_content' in locals(): print(f"Raw content (first 500 chars): {report_content[:500]}")
        return pd.DataFrame()

# --- Fiscal Quarter Calculation Function ---
def get_fiscal_quarter_info(date_input):
    if pd.isnull(date_input): return pd.NaT, None, None
    
    date_to_process = date_input
    if isinstance(date_to_process, str): date_to_process = pd.to_datetime(date_to_process, errors='coerce')
    elif not isinstance(date_to_process, pd.Timestamp):
        try: date_to_process = pd.to_datetime(date_to_process, errors='coerce')
        except Exception: return pd.NaT, None, None
    
    if pd.NaT is date_to_process : return pd.NaT, None, None 

    year, month = date_to_process.year, date_to_process.month
    fiscal_year_label_num, quarter_start_date, quarter_label_str = None, pd.NaT, None

    if month == 1:
        quarter_start_date, fiscal_year_label_num, quarter_label_str = pd.Timestamp(year=year-1, month=11, day=1), year, f'FY{year}Q4'
    elif month in [2, 3, 4]:
        quarter_start_date, fiscal_year_label_num, quarter_label_str = pd.Timestamp(year=year, month=2, day=1), year + 1, f'FY{year+1}Q1'
    elif month in [5, 6, 7]:
        quarter_start_date, fiscal_year_label_num, quarter_label_str = pd.Timestamp(year=year, month=5, day=1), year + 1, f'FY{year+1}Q2'
    elif month in [8, 9, 10]:
        quarter_start_date, fiscal_year_label_num, quarter_label_str = pd.Timestamp(year=year, month=8, day=1), year + 1, f'FY{year+1}Q3'
    elif month in [11, 12]:
        quarter_start_date, fiscal_year_label_num, quarter_label_str = pd.Timestamp(year=year, month=11, day=1), year + 1, f'FY{year+1}Q4'
    
    return quarter_start_date, quarter_label_str, fiscal_year_label_num

print("Cell 3: Utility Functions - Defined")

Cell 3: Utility Functions - Defined


In [4]:
# Authenticate to Salesforce
sf_session = None # Initialize
try:
    sf_session = Salesforce(username=SF_USERNAME, password=SF_PASSWORD, security_token=SF_SECURITY_TOKEN)
    print("Salesforce authentication successful.")
except Exception as e:
    print(f"Salesforce authentication failed: {e}")
    sf_session = None # Ensure it's None on failure

# Download the main dataframe
raw_df = pd.DataFrame() # Initialize
if sf_session:
    raw_df = download_sf_report(SF_INSTANCE_URL, MAIN_REPORT_ID, sf_session)
else:
    print("Skipping report download due to authentication failure.")

if raw_df.empty:
    print("WARNING: Raw DataFrame is empty. Subsequent cells may fail or produce no results.")
else:
    print(f"Raw DataFrame loaded with shape: {raw_df.shape}")

Salesforce authentication successful.
Report 00OUO000009IZVD2A4 downloaded successfully. Shape: (14718, 35)
Raw DataFrame loaded with shape: (14718, 35)


In [5]:
# ## Data Preprocessing
#
# ### Key Steps:
# 1.  **Date Conversion:** Convert all relevant date columns to a proper datetime format.
# 2.  **ARR Processing:** Use `'ARR Change'` as the primary value metric. If it's not available, fall back to `'ARR'` or create a zero-value column to prevent errors. This ensures the notebook is robust.
# 3.  **Fiscal Period Calculation:** For each opportunity, calculate its fiscal quarter based on both its `Created Date` and `Close Date` using the `get_fiscal_quarter_info` utility.
# 4.  **In-Quarter Flag:** Create a boolean flag, `'Inquarter Booking Flag'`, which is `True` if an opportunity's creation and close fiscal quarters are the same.
# 5.  **Opportunity ID:** Dynamically find the correct Opportunity ID column from a list of potential names to make the notebook more adaptable to different report formats.
# 6.  **In-Quarter DataFrame (`inq_df`):** Create a separate DataFrame containing only the deals flagged as in-quarter bookings for focused analysis and forecasting.
# 7.  **Segment Filtering:** Filter out Self Serve, Self Service, and Unknown segments from analysis.

processed_df = pd.DataFrame()
inq_df = pd.DataFrame()
OPP_ID_COL_NAME = None 

if not raw_df.empty:
    processed_df = raw_df.copy()

    # 1. Convert Date Columns
    date_cols_to_convert = ['Created Date', 'Close Date', 'Renewal Date', 'Opportunity Created Date']
    for col in date_cols_to_convert:
        if col in processed_df.columns:
            processed_df[col] = pd.to_datetime(processed_df[col], errors='coerce')

    # 2. Process ARR Column (Using 'ARR Change')
    if 'ARR Change' in processed_df.columns:
        processed_df['ARR Change'] = pd.to_numeric(processed_df['ARR Change'], errors='coerce').fillna(0)
        print("Info: 'ARR Change' column processed as the primary value metric.")
    else:
        # If 'ARR Change' is missing, fall back to 'ARR' or create a zero column.
        if 'ARR' in processed_df.columns:
            print("Warning: 'ARR Change' column not found. Falling back to using 'ARR'.")
            processed_df.rename(columns={'ARR': 'ARR Change'}, inplace=True) # Rename for consistency downstream
            processed_df['ARR Change'] = pd.to_numeric(processed_df['ARR Change'], errors='coerce').fillna(0)
        else:
            print("CRITICAL WARNING: Neither 'ARR Change' nor 'ARR' column found. Creating 'ARR Change' with zeros.")
            processed_df['ARR Change'] = 0

    # 3. Derive Fiscal Period Columns for the ENTIRE dataset
    created_date_source_col = next((col for col in ['Opportunity Created Date', 'Created Date'] if col in processed_df.columns and processed_df[col].notnull().any()), None)
    if created_date_source_col:
        processed_df['Fiscal Period - Created Date'] = processed_df[created_date_source_col].apply(lambda x: get_fiscal_quarter_info(x)[1])
    if 'Close Date' in processed_df.columns and processed_df['Close Date'].notnull().any():
        processed_df['Fiscal Period - Corrected'] = processed_df['Close Date'].apply(lambda x: get_fiscal_quarter_info(x)[1])

    # 4. Create In-Quarter Booking Flag based on the full dataset
    required_fiscal_cols = ['Fiscal Period - Corrected', 'Fiscal Period - Created Date']
    if all(col in processed_df.columns and processed_df[col].notnull().any() for col in required_fiscal_cols):
        processed_df['Inquarter Booking Flag'] = processed_df['Fiscal Period - Corrected'] == processed_df['Fiscal Period - Created Date']
        print("Info: 'Inquarter Booking Flag' calculated on the full dataset.")
    else:
        print("Warning: Could not create 'Inquarter Booking Flag' due to missing fiscal period columns.")
        processed_df['Inquarter Booking Flag'] = False

    # 5. Identify Opportunity ID Column
    potential_opp_id_cols = ['Opportunity ID', 'SFDC ID 18 Digit', 'Opportunity: ID'] 
    OPP_ID_COL_NAME = next((col for col in potential_opp_id_cols if col in processed_df.columns), None)
    if OPP_ID_COL_NAME is None and not processed_df.empty:
        processed_df['opportunity_pseudo_id'] = range(len(processed_df))
        OPP_ID_COL_NAME = 'opportunity_pseudo_id'
    
    # 6. Filter out unwanted segments
    segments_to_exclude = ['Self Serve', 'Self Service', 'Unknown']
    if 'Segment - historical' in processed_df.columns:
        original_count = len(processed_df)
        processed_df = processed_df[~processed_df['Segment - historical'].isin(segments_to_exclude)]
        filtered_count = len(processed_df)
        excluded_count = original_count - filtered_count
        print(f"Segment filtering: Excluded {excluded_count} records from segments: {segments_to_exclude}")
        print(f"Remaining records after segment filtering: {filtered_count}")
    else:
        print("Warning: 'Segment - historical' column not found. Skipping segment filtering.")
    
    # 7. Create inq_df (from the filtered dataset)
    inq_df = processed_df[processed_df['Inquarter Booking Flag']].copy() if 'Inquarter Booking Flag' in processed_df.columns else pd.DataFrame()
    
    print(f"\nPreprocessing complete on filtered dataset. Shape: {processed_df.shape}")
    if OPP_ID_COL_NAME: print(f"Opportunity ID Column used: {OPP_ID_COL_NAME}")
    if not inq_df.empty: print(f"In-Quarter DataFrame (inq_df) shape: {inq_df.shape}")
else:
    print("Preprocessing skipped: Raw DataFrame is empty.")

# Create sf alias for sf_session to fix pipegen cell
if 'sf_session' in locals():
    sf = sf_session

print("Cell 5: Data Preprocessing with Segment Filtering - Completed Successfully")

Info: 'ARR Change' column processed as the primary value metric.
Info: 'Inquarter Booking Flag' calculated on the full dataset.
Segment filtering: Excluded 148 records from segments: ['Self Serve', 'Self Service', 'Unknown']
Remaining records after segment filtering: 14570

Preprocessing complete on filtered dataset. Shape: (14570, 36)
Opportunity ID Column used: SFDC ID 18 Digit
In-Quarter DataFrame (inq_df) shape: (5971, 36)
Cell 5: Data Preprocessing with Segment Filtering - Completed Successfully


In [6]:
# ## Date Enhancement System (Using Production Module)
#
# This section uses the modular date_utils.py to enhance date columns with production logic.

from date_utils import enhance_dataframe_dates

print("--- Date Enhancement Using Production Module ---")

# Safety check: Ensure processed_df exists before proceeding
if 'processed_df' not in locals() or processed_df.empty:
    print("❌ ERROR: processed_df is not available. Please run the Data Preprocessing cell first.")
    print("   The Date Enhancement system requires processed_df to be created from the raw Salesforce data.")
else:
    print("✅ processed_df found successfully - proceeding with date enhancement")
    
    # Use the modular function to enhance dates
    processed_df = enhance_dataframe_dates(processed_df)
    
    print("--- Finished Date Enhancement System Implementation ---")

--- Date Enhancement Using Production Module ---
✅ processed_df found successfully - proceeding with date enhancement
Enhancing date columns: ['Created Date', 'Close Date', 'SQO Date', 'SAO Date', 'Timestamp: Solution Validation']
Converted Created Date to datetime
Converted Close Date to datetime
Converted SQO Date to datetime
Converted SAO Date to datetime
Converted Timestamp: Solution Validation to datetime
Processing date breakdowns for: Created Date
Added breakdown columns for Created Date: ['Created Date_Quarter', 'Created Date_Week_of_Quarter', 'Created Date_Month', 'Created Date_Day_of_Week', 'Created Date_Day_Name']
Processing date breakdowns for: Close Date
Added breakdown columns for Close Date: ['Close Date_Quarter', 'Close Date_Week_of_Quarter', 'Close Date_Month', 'Close Date_Day_of_Week', 'Close Date_Day_Name']
Processing date breakdowns for: SQO Date
Added breakdown columns for SQO Date: ['SQO Date_Quarter', 'SQO Date_Week_of_Quarter', 'SQO Date_Month', 'SQO Date_Day_of

In [7]:
# ## ARR Change History Processing and Pipeline Generation Analysis (Direct Load)
#
# This section loads ARR change history directly from Salesforce and processes it for pipeline generation analysis.

from pipegen_analyzer import process_arr_change_history, display_pipegen_summary, calculate_pipegen_6_row_analysis

print("🚀 ARR CHANGE HISTORY & PIPELINE GENERATION ANALYSIS")
print("=" * 60)

# Check if we have Salesforce connection
if 'sf' in locals():
    print("✅ Salesforce connection found")
    
    try:
        # Direct report loading approach with correct report ID
        sf_instance = 'https://envoy.my.salesforce.com/'  # Your Salesforce Instance URL
        reportId = '00OUO000009jhTS2AY'  # ARR change history report ID (corrected)
        export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
        sfUrl = sf_instance + reportId + export
        
        print(f"📊 Loading ARR change history from report: {reportId}")
        response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
        response.raise_for_status()
        
        download_report = response.content.decode('utf-8')
        raw_pipegen_df = pd.read_csv(StringIO(download_report))
        
        print(f"✅ Successfully loaded {len(raw_pipegen_df)} records from ARR change history report")
        
        # Process the raw data for pipeline generation analysis with stage timestamp data
        stage_timestamp_data = processed_df if 'processed_df' in locals() and not processed_df.empty else None
        if stage_timestamp_data is not None:
            print(f"🔗 Using stage timestamp data from processed_df ({len(stage_timestamp_data)} opportunities)")
        else:
            print("⚠️ No stage timestamp data available (processed_df not found)")
            
        pipegen_df = process_arr_change_history(raw_pipegen_df, stage_timestamp_data)
        
        if not pipegen_df.empty:
            # Display summary
            display_pipegen_summary(pipegen_df)
            
            print(f"\n🎯 Pipegen DataFrame created with {len(pipegen_df)} opportunities")
            print(f"💰 Total pipeline generated: ${pipegen_df['Pipegen ARR'].sum():,.0f}")
            
            # Display sample data
            print(f"\n📋 Sample Pipegen Data:")
            sample_cols = ['SFDC ID 18 Digit', 'Pipegen ARR', 'SAO Date_Quarter', 'Segment', 'Stage']
            available_cols = [col for col in sample_cols if col in pipegen_df.columns]
            if available_cols:
                display(pipegen_df[available_cols].head(10))
            
        else:
            print("❌ No pipeline generation data could be processed after loading report")
            pipegen_df = pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error loading ARR change history report: {e}")
        print("📝 Continuing analysis without Pipegen data - all other analyses will work")
        pipegen_df = pd.DataFrame()
        import traceback
        traceback.print_exc()
        
else:
    print("❌ No Salesforce connection available. Please run the Salesforce authentication cell first.")
    pipegen_df = pd.DataFrame()  # Create empty DataFrame as fallback

🚀 ARR CHANGE HISTORY & PIPELINE GENERATION ANALYSIS
✅ Salesforce connection found
📊 Loading ARR change history from report: 00OUO000009jhTS2AY
✅ Successfully loaded 28849 records from ARR change history report
🔗 Using stage timestamp data from processed_df (14570 opportunities)
🔄 Processing ARR change history for pipeline generation analysis...
📋 Available columns in ARR change history: ['SFDC ID 18 Digit', 'ARR Change', 'Old Value', 'Field / Event', 'New Value', 'Edit Date']


/Users/bchen/Documents/Master reporting automation/pipegen_analyzer.py:89: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  history_df[col] = pd.to_datetime(history_df[col], errors='coerce')


📊 Processing 28849 ARR change history records...
🔄 Consolidating multiple daily edits per opportunity...
📉 Consolidated to 13796 records after removing multiple daily edits
🔗 Merging ARR change history with stage timestamp data...
📋 Available columns in stage timestamp data: ['SFDC ID 18 Digit', 'Opportunity Name', 'Account Name', 'Account 18 Digit ID', 'Opportunity Owner', 'Credited Rep', 'Credited Rep Role', 'Owner User Segment', 'Owner Role', 'Stage', 'Forecast Category', 'Fiscal Period - Created Date', 'Fiscal Period - Corrected', 'Created Date', 'SQO Date', 'SAO Date', 'Timestamp: Solution Validation', 'Close Date', 'Source', 'Bookings Type', 'Type', 'Primary Partner', 'Won', 'Age', 'Days Since Qualified', 'ARR Change', 'Workplace ARR', 'VR ARR', 'Emergency Notification ARR', 'Deliveries ARR', 'Priority Support ARR', 'Segment - historical', 'Created By Group - Historical', 'Created By Role - Historical', 'Opportunity Owner: Active', 'Inquarter Booking Flag', 'Created Date_Quarter'

,SFDC ID 18 Digit,Pipegen ARR,SAO Date_Quarter,Segment,Stage
0,006UO00000G2NXCYA3,"1,177.20",4,SMB,Closed Won
1,006UO00000NwzU5YAJ,"1,177.20",1,SMB,Closed Lost
2,006UO00000GNIw0YAH,"7,177.20",4,SMB,Closed Lost
3,006UO000001VYJQYA4,"1,236.29",4,Enterprise,Closed Lost
4,006UO000002c1QFYAY,"3,847.18",4,Enterprise,Closed Lost
5,006UO000005N1a1YAC,274.80,1,Enterprise,Closed Won
6,006UO00000JZiZXYA1,"6,316.81",1,Enterprise,Closed Lost
7,0065b000013y7dTAAQ,"2,700.00",3,SMB,Closed Won
8,0065b000013y0VkAAI,"2,199.04",3,Mid Market,Closed Won
9,006UO00000FSz7xYAD,-0.06,3,Mid Market,Closed Lost


In [8]:
# ## Pipegen Analysis - 6 Row Structure with Quarterly Breakdown
#
# This section performs 6-row pipeline generation analysis with quarterly columns and comparison metrics.

from metrics_calculator import calculate_pipegen_quarterly_summary, add_pipegen_comparison_columns

print("🎯 PIPEGEN QUARTERLY ANALYSIS - 6 ROW STRUCTURE")
print("=" * 55)

# Check if pipegen_df exists from previous cell
if 'pipegen_df' in locals() and not pipegen_df.empty:
    # Calculate quarterly summary with 6-row structure using fiscal periods
    # Use 'Bookings Type' column which contains 'Expansion' and 'New Business' values
    pipegen_quarterly_summary = calculate_pipegen_quarterly_summary(
        pipegen_df, 
        bookings_type_col='Bookings Type', 
        start_fiscal_year=START_FISCAL_YEAR_FOR_ANALYSIS if 'START_FISCAL_YEAR_FOR_ANALYSIS' in locals() else 2023
    )
    
    if not pipegen_quarterly_summary.empty:
        # Add comparison columns (QoQ, vs Last 4Q Avg, YoY)
        pipegen_summary_with_comparisons = add_pipegen_comparison_columns(pipegen_quarterly_summary)
        
        print("\n✅ Pipegen quarterly analysis completed successfully!")
        print(f"\nDisplaying: Pipeline Generation Summary (Fiscal Period Breakdown)")
        
        # Display the full quarterly breakdown with comparisons
        display(pipegen_summary_with_comparisons)
        
        print("\n📊 Summary Statistics:")
        if 'Total Pipeline Generated' in pipegen_summary_with_comparisons.index:
            total_pipegen = pipegen_summary_with_comparisons.loc['Total Pipeline Generated'].iloc[0]
            print(f"Total Pipeline Generated: ${total_pipegen:,.0f}")
            
            # Get most recent quarter value
            quarter_cols = [col for col in pipegen_summary_with_comparisons.columns 
                          if isinstance(col, str) and col.startswith('FY') and 'Q' in col]
            if quarter_cols:
                latest_quarter = sorted(quarter_cols)[-1]
                latest_quarter_total = pipegen_summary_with_comparisons.loc['Total InQuarter', latest_quarter]
                print(f"Latest Quarter ({latest_quarter}) Pipeline: ${latest_quarter_total:,.0f}")
    else:
        print("❌ Could not generate pipegen quarterly summary - no data available")
        
else:
    print("❌ Pipegen data not found. Please run the ARR Change History cell first.")
    print("   This analysis requires pipegen_df to be created from the Salesforce ARR change history.")

🎯 PIPEGEN QUARTERLY ANALYSIS - 6 ROW STRUCTURE
📊 Sample fiscal period mapping from SAO Date:     SAO Date  SAO Date_Quarter Fiscal Period
0 2024-11-08                 4      FY2025Q4
1 2025-04-28                 1      FY2026Q1
2 2024-11-19                 4      FY2025Q4
3 2023-12-05                 4      FY2024Q4
4 2024-01-04                 4      FY2024Q4
📊 Found fiscal periods in pipegen data: ['FY2023Q2', 'FY2023Q3', 'FY2023Q4', 'FY2024Q1', 'FY2024Q2', 'FY2024Q3', 'FY2024Q4', 'FY2025Q1', 'FY2025Q2', 'FY2025Q3', 'FY2025Q4', 'FY2026Q1', 'FY2026Q2']
📊 Available Bookings Type values in pipegen data: ['New Business' 'Expansion']
📊 Looking for Bookings Type values: 'Expansion' and 'New Business'
📊 Expansion data found: 3697 opportunities
📊 New Business data found: 2383 opportunities

✅ Pipegen quarterly analysis completed successfully!

Displaying: Pipeline Generation Summary (Fiscal Period Breakdown)


,FY2023Q2,FY2023Q3,FY2023Q4,FY2024Q1,FY2024Q2,FY2024Q3,FY2024Q4,FY2025Q1,FY2025Q2,FY2025Q3,FY2025Q4,FY2026Q1,FY2026Q2,QoQ Change,vs Last 4Q Avg,YoY Change
Metric,,,,,,,,,,,,,,,,
Expansion,"1,000.00","38,470.38","133,784.00","192,196.39","490,672.57","2,758,510.75","4,095,681.41","6,062,623.75","5,465,500.76","4,257,817.47","3,367,287.30","8,606,602.93","5,703,170.50",-33.73,5.14,4.35
New Business,"299,064.00","56,598.08","46,583.52","1,005,257.16","829,984.48","3,820,622.40","3,830,641.33","4,664,216.67","4,641,843.93","3,404,378.52","3,976,450.36","7,304,096.56","5,210,567.71",-28.66,7.84,12.25
Total by Period,"300,064.00","95,068.46","180,367.52","1,197,453.55","1,320,657.05","6,579,133.15","7,926,322.74","10,726,840.42","10,107,344.69","7,662,195.99","7,343,737.66","15,910,699.49","10,913,738.21",-31.41,6.41,7.98



📊 Summary Statistics:


In [9]:
# ## Enhanced Comprehensive Trend Analysis (Using Module)
#
# This section performs comprehensive trend analysis including Pipeline Generation metrics using the modular approach.

from trend_analyzer import perform_trend_analysis, save_trend_results

print("🚀 COMPREHENSIVE TREND ANALYSIS WITH PIPEGEN SUPPORT")
print("=" * 60)

# Check if required data exists
required_vars = ['processed_df', 'OPP_ID_COL_NAME', 'START_FISCAL_YEAR_FOR_ANALYSIS']
missing_vars = [var for var in required_vars if var not in locals()]

if missing_vars:
    print(f"❌ Missing required variables: {missing_vars}")
    print("   Please run the Data Preprocessing cell first.")
else:
    # Prepare inq_df if it exists
    inq_df_for_analysis = inq_df if 'inq_df' in locals() and not inq_df.empty else pd.DataFrame()
    
    # Prepare pipegen_df if it exists
    pipegen_df_for_analysis = pipegen_df if 'pipegen_df' in locals() and not pipegen_df.empty else None
    
    # Define custom thresholds (optional - will use defaults if None)
    custom_thresholds = {
        'pct_change_threshold': 20.0,
        'abs_change_threshold_arr': 100000,
        'abs_change_threshold_opps': 10,
        'abs_change_threshold_wr': 10.0,
        'abs_change_threshold_asc': 30.0,
        'abs_change_threshold_asp': 5000,
        'abs_change_threshold_pipegen': 50000
    }
    
    # Perform trend analysis
    significant_changes_df = perform_trend_analysis(
        processed_df=processed_df,
        inq_df=inq_df_for_analysis,
        opp_id_col_name=OPP_ID_COL_NAME,
        start_fiscal_year=START_FISCAL_YEAR_FOR_ANALYSIS,
        stage_won=STAGE_WON if 'STAGE_WON' in locals() else 'Closed Won',
        pipegen_df=pipegen_df_for_analysis,
        thresholds=custom_thresholds
    )
    
    # Save results if any significant changes found
    if not significant_changes_df.empty:
        save_trend_results(significant_changes_df, 'enhanced_trends_summary_with_pipegen.csv')
        print("\n🎉 Enhanced trend analysis with Pipegen support completed!")
    else:
        print("\n📊 Trend analysis completed - no significant changes detected.")

print("\n--- Trend Analysis Module Implementation Complete ---")

🚀 COMPREHENSIVE TREND ANALYSIS WITH PIPEGEN SUPPORT
❌ Missing required variables: ['processed_df', 'OPP_ID_COL_NAME', 'START_FISCAL_YEAR_FOR_ANALYSIS']
   Please run the Data Preprocessing cell first.

--- Trend Analysis Module Implementation Complete ---


In [10]:
asdfadfsadfs

NameError: name 'asdfadfsadfs' is not defined

In [ ]:
# ## Data Preprocessing
#
# ### Key Steps:
# 1.  **Date Conversion:** Convert all relevant date columns to a proper datetime format.
# 2.  **ARR Processing:** Use `'ARR Change'` as the primary value metric. If it's not available, fall back to `'ARR'` or create a zero-value column to prevent errors. This ensures the notebook is robust.
# 3.  **Fiscal Period Calculation:** For each opportunity, calculate its fiscal quarter based on both its `Created Date` and `Close Date` using the `get_fiscal_quarter_info` utility.
# 4.  **In-Quarter Flag:** Create a boolean flag, `'Inquarter Booking Flag'`, which is `True` if an opportunity's creation and close fiscal quarters are the same.
# 5.  **Opportunity ID:** Dynamically find the correct Opportunity ID column from a list of potential names to make the notebook more adaptable to different report formats.
# 6.  **In-Quarter DataFrame (`inq_df`):** Create a separate DataFrame containing only the deals flagged as in-quarter bookings for focused analysis and forecasting.

processed_df = pd.DataFrame()
inq_df = pd.DataFrame()
OPP_ID_COL_NAME = None 

if not raw_df.empty:
    processed_df = raw_df.copy()

    # 1. Convert Date Columns
    date_cols_to_convert = ['Created Date', 'Close Date', 'Renewal Date', 'Opportunity Created Date']
    for col in date_cols_to_convert:
        if col in processed_df.columns:
            processed_df[col] = pd.to_datetime(processed_df[col], errors='coerce')

    # 2. Process ARR Column (Using 'ARR Change')
    if 'ARR Change' in processed_df.columns:
        processed_df['ARR Change'] = pd.to_numeric(processed_df['ARR Change'], errors='coerce').fillna(0)
        print("Info: 'ARR Change' column processed as the primary value metric.")
    else:
        # If 'ARR Change' is missing, fall back to 'ARR' or create a zero column.
        if 'ARR' in processed_df.columns:
            print("Warning: 'ARR Change' column not found. Falling back to using 'ARR'.")
            processed_df.rename(columns={'ARR': 'ARR Change'}, inplace=True) # Rename for consistency downstream
            processed_df['ARR Change'] = pd.to_numeric(processed_df['ARR Change'], errors='coerce').fillna(0)
        else:
            print("CRITICAL WARNING: Neither 'ARR Change' nor 'ARR' column found. Creating 'ARR Change' with zeros.")
            processed_df['ARR Change'] = 0

    # 3. Derive Fiscal Period Columns for the ENTIRE dataset
    created_date_source_col = next((col for col in ['Opportunity Created Date', 'Created Date'] if col in processed_df.columns and processed_df[col].notnull().any()), None)
    if created_date_source_col:
        processed_df['Fiscal Period - Created Date'] = processed_df[created_date_source_col].apply(lambda x: get_fiscal_quarter_info(x)[1])
    if 'Close Date' in processed_df.columns and processed_df['Close Date'].notnull().any():
        processed_df['Fiscal Period - Corrected'] = processed_df['Close Date'].apply(lambda x: get_fiscal_quarter_info(x)[1])

    # 4. Create In-Quarter Booking Flag based on the full dataset
    required_fiscal_cols = ['Fiscal Period - Corrected', 'Fiscal Period - Created Date']
    if all(col in processed_df.columns and processed_df[col].notnull().any() for col in required_fiscal_cols):
        processed_df['Inquarter Booking Flag'] = processed_df['Fiscal Period - Corrected'] == processed_df['Fiscal Period - Created Date']
        print("Info: 'Inquarter Booking Flag' calculated on the full dataset.")
    else:
        print("Warning: Could not create 'Inquarter Booking Flag' due to missing fiscal period columns.")
        processed_df['Inquarter Booking Flag'] = False

    # 5. Identify Opportunity ID Column
    potential_opp_id_cols = ['Opportunity ID', 'SFDC ID 18 Digit', 'Opportunity: ID'] 
    OPP_ID_COL_NAME = next((col for col in potential_opp_id_cols if col in processed_df.columns), None)
    if OPP_ID_COL_NAME is None and not processed_df.empty:
        processed_df['opportunity_pseudo_id'] = range(len(processed_df))
        OPP_ID_COL_NAME = 'opportunity_pseudo_id'
    
    # 6. Create inq_df (still from the full dataset)
    inq_df = processed_df[processed_df['Inquarter Booking Flag']].copy() if 'Inquarter Booking Flag' in processed_df.columns else pd.DataFrame()
    
    print(f"\nPreprocessing complete on full raw dataset. Shape: {processed_df.shape}")
    if OPP_ID_COL_NAME: print(f"Opportunity ID Column used: {OPP_ID_COL_NAME}")
    if not inq_df.empty: print(f"In-Quarter DataFrame (inq_df) shape: {inq_df.shape}")
else:
    print("Preprocessing skipped: Raw DataFrame is empty.")

# ## Trend Analysis Functions
#
# This cell defines all necessary functions for calculating metrics and identifying significant trends. This modular approach allows for flexible comparisons (QoQ, YoY, etc.) without duplicating code.

def calculate_quarterly_metrics(df_full, opp_id_col_name, group_by_cols=None):
    """Calculates all key sales metrics on a quarterly basis, with optional grouping."""
    if df_full.empty: return pd.DataFrame()
    required_cols = ['Fiscal Period - Corrected', 'ARR Change', 'Stage', 'Created Date', 'Close Date', opp_id_col_name]
    if not all(col in df_full.columns for col in required_cols):
        print(f"Warning: Missing one or more required columns in calculate_quarterly_metrics. Needed: {required_cols}")
        return pd.DataFrame()
    df_copy = df_full.copy()
    df_won = df_copy[df_copy['Stage'] == STAGE_WON].copy()
    if not df_won.empty:
        df_won.loc[:, 'Sales Cycle Days'] = (df_won['Close Date'] - df_won['Created Date']).dt.days
        df_copy = df_copy.merge(df_won[['Sales Cycle Days']], left_index=True, right_index=True, how='left')
    else:
        df_copy['Sales Cycle Days'] = pd.NA
    grouping_cols = []
    if group_by_cols:
        grouping_cols.extend([group_by_cols] if isinstance(group_by_cols, str) else list(group_by_cols))
    grouping_cols.append('Fiscal Period - Corrected')
    agg_funcs = {
        'Total ARR': ('ARR Change', 'sum'),
        '# of Opps': (opp_id_col_name, 'nunique'),
        '# of Won Opps': (opp_id_col_name, lambda x: x[df_copy.loc[x.index, 'Stage'] == STAGE_WON].nunique()),
        'Total Relevant Opps for WR': (opp_id_col_name, lambda x: x[df_copy.loc[x.index, 'Stage'].isin([STAGE_WON, STAGE_LOST])].nunique()),
        'Won ARR': ('ARR Change', lambda x: x[df_copy.loc[x.index, 'Stage'] == STAGE_WON].sum()),
        'Avg Sales Cycle': ('Sales Cycle Days', 'mean')
    }
    quarterly_summary = df_copy.groupby(grouping_cols, observed=False).agg(**agg_funcs)
    quarterly_summary['Avg Sales Price'] = (quarterly_summary['Won ARR'] / quarterly_summary['# of Won Opps'].replace(0, np.nan)).fillna(0)
    quarterly_summary['Win Rate (Count)'] = (quarterly_summary['# of Won Opps'] / quarterly_summary['Total Relevant Opps for WR'].replace(0, np.nan) * 100).fillna(0)
    final_metrics = ['Total ARR', '# of Opps', '# of Won Opps', 'Won ARR', 'Avg Sales Price', 'Win Rate (Count)', 'Avg Sales Cycle']
    return quarterly_summary[final_metrics]

def build_comparison_view(quarterly_metrics_df, current_q, comparison_q_data, comparison_label, group_tuple=None):
    """Constructs a side-by-side comparison DataFrame for a given metric set."""
    if group_tuple:
        # For grouped data, select the group first, then the specific quarter
        df_to_use = quarterly_metrics_df.loc[group_tuple]
        if current_q not in df_to_use.index: return pd.DataFrame()
        current_q_data = df_to_use.loc[current_q]
    else:
        # For overall data, the index is just the quarter
        if current_q not in quarterly_metrics_df.index: return pd.DataFrame()
        current_q_data = quarterly_metrics_df.loc[current_q]
    comparison_view = pd.DataFrame({'Current Quarter': current_q_data, comparison_label: comparison_q_data}).T
    abs_change = comparison_view.loc['Current Quarter'] - comparison_view.loc[comparison_label]
    pct_change = (abs_change / comparison_view.loc[comparison_label].replace(0, np.nan) * 100)
    comparison_view.loc['Absolute Change'] = abs_change
    comparison_view.loc['Percent Change'] = pct_change
    return comparison_view.T.fillna({'Avg Sales Cycle': pd.NA}).fillna(0)

def identify_and_report_significant_changes(comparison_df, current_q, comparison_label, group_label="Overall"):
    """Analyzes a comparison DataFrame and returns a list of significant findings based on thresholds."""
    significant_findings_list = []
    if comparison_df.empty: return significant_findings_list
    
    METRICS_TO_ANALYZE = ['Total ARR', '# of Opps', '# of Won Opps', 'Avg Sales Price', 'Win Rate (Count)', 'Avg Sales Cycle']
    for metric_name in METRICS_TO_ANALYZE:
        if metric_name not in comparison_df.index: continue
        row_data = comparison_df.loc[metric_name]
        pct_val, absolute_val = row_data.get('Percent Change', 0), row_data.get('Absolute Change', 0)
        is_sig_drop, is_sig_inc = False, False
        abs_num, pct_num = pd.to_numeric(absolute_val, 'coerce'), pd.to_numeric(pct_val, 'coerce')

        if metric_name == 'Total ARR':
            is_sig_drop = (pct_num <= PCT_CHANGE_THRESHOLD_DROP and abs(abs_num or 0) >= ABS_ARR_CHANGE_THRESHOLD)
            is_sig_inc = (pct_num >= PCT_CHANGE_THRESHOLD_INCREASE and abs(abs_num or 0) >= ABS_ARR_CHANGE_THRESHOLD)
        elif metric_name in ['# of Opps', '# of Won Opps']:
            is_sig_drop = (pct_num <= PCT_CHANGE_THRESHOLD_DROP and abs(abs_num or 0) >= ABS_OPP_CHANGE_THRESHOLD)
            is_sig_inc = (pct_num >= PCT_CHANGE_THRESHOLD_INCREASE and abs(abs_num or 0) >= ABS_OPP_CHANGE_THRESHOLD)
        elif metric_name == 'Avg Sales Price':
            is_sig_drop = (pct_num <= PCT_CHANGE_THRESHOLD_DROP and abs(abs_num or 0) >= ASP_CHANGE_THRESHOLD)
            is_sig_inc = (pct_num >= PCT_CHANGE_THRESHOLD_INCREASE and abs(abs_num or 0) >= ASP_CHANGE_THRESHOLD)
        elif metric_name == 'Win Rate (Count)':
            is_sig_drop = (abs_num <= -WIN_RATE_PCT_POINT_CHANGE_THRESHOLD)
            is_sig_inc = (abs_num >= WIN_RATE_PCT_POINT_CHANGE_THRESHOLD)
        elif metric_name == 'Avg Sales Cycle':
            is_sig_drop = ((abs_num <= -SALES_CYCLE_DAY_CHANGE_THRESHOLD) or (pct_num <= -SALES_CYCLE_PCT_CHANGE_THRESHOLD))
            is_sig_inc = ((abs_num >= SALES_CYCLE_DAY_CHANGE_THRESHOLD) or (pct_num >= SALES_CYCLE_PCT_CHANGE_THRESHOLD))
        
        if is_sig_drop or is_sig_inc:
            significant_findings_list.append({
                'Quarter': current_q, 'Comparison': comparison_label, 'Group': group_label, 'Metric': metric_name, 
                '% Change': f"{pct_num:.2f}%" if pd.notnull(pct_num) else "N/A", 
                'Abs Change': f"{abs_num:,.2f}" if pd.notnull(abs_num) else "N/A",
                'Type': 'Drop/Decrease' if is_sig_drop else 'Increase'
            })
    return significant_findings_list

print("Cell 7: All Trend Analysis Functions Defined")


In [ ]:
import pandas as pd
import numpy as np
# 1. --- ADDED --- Import the display function for rich output in notebooks
from IPython.display import display

# ## Main Summary Tables (Similar to CSV Output)
#
# **NOTE:** "Total bookings" is the sum of 'ARR Change' for 'Closed Won' deals.
# "Total Inquarter" is the sum of 'ARR Change' for 'Closed Won' deals that were also created in the same quarter.

# --- OPTIONAL BUT RECOMMENDED --- Configure pandas for better display
# This improves readability of the numbers in your tables.
pd.options.display.float_format = '{:,.2f}'.format

def add_comparison_columns(df):
    """
    Add QoQ Change, vs Last 4 Quarter Average, and YoY Change columns
    to the rightmost side of the dataframe
    """
    if df.empty:
        return df
    
    # Get fiscal quarter columns (exclude non-quarter columns)
    quarter_cols = [col for col in df.columns if isinstance(col, str) and col.startswith('FY') and 'Q' in col]
    quarter_cols.sort()
    
    if len(quarter_cols) < 2:
        return df
    
    # Initialize comparison columns
    qoq_change_col = 'QoQ Change'
    vs_last4_avg_col = 'vs Last 4Q Avg'
    yoy_change_col = 'YoY Change'
    
    # Get the most recent quarter (last column)
    current_quarter = quarter_cols[-1]
    
    # Convert columns to numeric, replacing non-numeric values with NaN
    for col in quarter_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # QoQ Change
    if len(quarter_cols) >= 2:
        prev_quarter = quarter_cols[-2]
        qoq_values = ((df[current_quarter] - df[prev_quarter]) / df[prev_quarter].replace(0, np.nan) * 100)
        df[qoq_change_col] = qoq_values.round(2)
    else:
        df[qoq_change_col] = np.nan
    
    # vs Last 4 Quarter Average
    if len(quarter_cols) >= 5:
        last_4_quarters = quarter_cols[-5:-1]  # Get 4 quarters before current
        avg_last_4 = df[last_4_quarters].mean(axis=1)
        vs_avg_values = ((df[current_quarter] - avg_last_4) / avg_last_4.replace(0, np.nan) * 100)
        df[vs_last4_avg_col] = vs_avg_values.round(2)
    else:
        df[vs_last4_avg_col] = np.nan
    
    # YoY Change
    # Find same quarter from previous year
    current_year = int(current_quarter[2:6])
    current_q = current_quarter[-2:]
    prev_year_quarter = f"FY{current_year-1}{current_q}"
    
    if prev_year_quarter in quarter_cols:
        yoy_values = ((df[current_quarter] - df[prev_year_quarter]) / df[prev_year_quarter].replace(0, np.nan) * 100)
        df[yoy_change_col] = yoy_values.round(2)
    else:
        df[yoy_change_col] = np.nan
    
    return df

print("--- Attempting to Generate Main Summary Tables (ARR Focused) ---")

# Define the stage name for a won deal, consistent with Cell 2's configuration
if 'STAGE_WON' not in locals(): STAGE_WON = 'Closed Won'

# Ensure processed_df is valid and has necessary columns
if not processed_df.empty and \
   all(col in processed_df.columns for col in ['Fiscal Period - Corrected', 'ARR Change', 'Bookings Type', 'Inquarter Booking Flag', 'Stage']):

    # Create a DataFrame containing only "booked" (won) deals. This will be the basis for ALL bookings calculations.
    booked_deals_df = processed_df[processed_df['Stage'] == STAGE_WON].copy()
    
    if booked_deals_df.empty:
        print("WARNING: No deals with Stage 'Closed Won' found. All booking values will be zero.")
    
    print(f"\n{'='*15} Generating Main Summary Table: By Bookings Type {'='*15}")

    # Create a DataFrame of deals that are BOTH in-quarter AND booked/won.
    in_quarter_booked_deals_df = booked_deals_df[booked_deals_df['Inquarter Booking Flag'] == True].copy()
    
    if in_quarter_booked_deals_df.empty:
        print("Warning: No 'Closed Won' deals were found within the in-quarter dataset.")
    
    # Calculate In-Quarter ARR using the filtered in_quarter_booked_deals_df
    inq_arr_by_bt_unstacked = in_quarter_booked_deals_df.groupby(
        ['Bookings Type', 'Fiscal Period - Corrected'], observed=False 
    )['ARR Change'].sum().unstack(fill_value=0)

    # Calculate Total Bookings based ONLY on all Closed Won deals
    total_arr_by_fp_series = booked_deals_df.groupby(
        'Fiscal Period - Corrected', observed=False
    )['ARR Change'].sum()

    summary_by_bookings_type_final = pd.DataFrame() 
    if not inq_arr_by_bt_unstacked.empty or not total_arr_by_fp_series.empty: # Proceed if either has data
        # If in-quarter is empty but total is not, start with an empty DF with the right columns
        if inq_arr_by_bt_unstacked.empty:
            all_btypes = booked_deals_df['Bookings Type'].dropna().unique()
            all_quarters = booked_deals_df['Fiscal Period - Corrected'].dropna().unique()
            summary_by_bookings_type_final = pd.DataFrame(0, index=all_btypes, columns=all_quarters)
        else:
            summary_by_bookings_type_final = inq_arr_by_bt_unstacked.copy()

        summary_by_bookings_type_final.loc['Total Inquarter'] = summary_by_bookings_type_final.sum(axis=0)
        
        if not total_arr_by_fp_series.empty:
            summary_by_bookings_type_final.loc['Total bookings'] = total_arr_by_fp_series.reindex(summary_by_bookings_type_final.columns).fillna(0)
        else:
            summary_by_bookings_type_final.loc['Total bookings'] = 0 

        if 'Total Inquarter' in summary_by_bookings_type_final.index and 'Total bookings' in summary_by_bookings_type_final.index:
            percent_inquarter_bt_series = (summary_by_bookings_type_final.loc['Total Inquarter'] / summary_by_bookings_type_final.loc['Total bookings'].replace(0, np.nan) * 100).fillna(0)
            summary_by_bookings_type_final.loc['Percent inquarter'] = percent_inquarter_bt_series.apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "0.00%")
        else:
            summary_by_bookings_type_final.loc['Percent inquarter'] = "N/A"

    fy_start_str_display = f"FY{START_FISCAL_YEAR_FOR_ANALYSIS}Q1"
    
    if not summary_by_bookings_type_final.empty:
        cols_to_keep_bt = [col for col in summary_by_bookings_type_final.columns if isinstance(col, str) and col >= fy_start_str_display]
        summary_by_bookings_type_filtered = summary_by_bookings_type_final[sorted(cols_to_keep_bt)]
        
        # Add comparison columns for numeric rows only
        numeric_rows = summary_by_bookings_type_filtered.index[summary_by_bookings_type_filtered.index != 'Percent inquarter']
        summary_numeric = summary_by_bookings_type_filtered.loc[numeric_rows].copy()
        summary_numeric = add_comparison_columns(summary_numeric)
        
        # Recombine with percent row
        if 'Percent inquarter' in summary_by_bookings_type_filtered.index:
            percent_row = summary_by_bookings_type_filtered.loc[['Percent inquarter']].copy()
            # Add empty comparison columns to percent row
            for col in ['QoQ Change', 'vs Last 4Q Avg', 'YoY Change']:
                percent_row[col] = 'N/A'
            summary_by_bookings_type_filtered = pd.concat([summary_numeric, percent_row])
        else:
            summary_by_bookings_type_filtered = summary_numeric
        
        print(f"\nDisplaying: Summary by Bookings Type (Filtered to display from {fy_start_str_display})")
        # 2. --- CHANGED --- Use display() instead of print() for a rich, scrollable HTML table
        display(summary_by_bookings_type_filtered)
    else:
        print("Could not generate 'Summary by Bookings Type'.")
        
    summary_by_segment_bookings_type_final = pd.DataFrame()
    if 'Segment - historical' in processed_df.columns:
        print(f"\n\n{'='*15} Generating Main Summary Table: By Segment & Bookings Type (ARR Focused) {'='*15}")
        
        inq_arr_by_seg_bt_unstacked = in_quarter_booked_deals_df.groupby(
            ['Segment - historical', 'Bookings Type', 'Fiscal Period - Corrected'], observed=False
        )['ARR Change'].sum().unstack(fill_value=0)
        
        total_arr_by_seg_fp_unstacked = booked_deals_df.groupby(
            ['Segment - historical', 'Fiscal Period - Corrected'], observed=False
        )['ARR Change'].sum().unstack(fill_value=0)
        
        all_segments_summary_list = []
        if not total_arr_by_seg_fp_unstacked.empty: 
            sorted_fiscal_quarters_cols = sorted([q for q in processed_df['Fiscal Period - Corrected'].dropna().unique() if q is not None])
            for segment_name in sorted(processed_df['Segment - historical'].dropna().unique()):
                segment_summary_df = pd.DataFrame(columns=sorted_fiscal_quarters_cols) 
                if segment_name in inq_arr_by_seg_bt_unstacked.index.get_level_values('Segment - historical'):
                    segment_inq_arr_for_bt_df = inq_arr_by_seg_bt_unstacked.loc[segment_name]
                    if isinstance(segment_inq_arr_for_bt_df, pd.Series): 
                        segment_inq_arr_for_bt_df = segment_inq_arr_for_bt_df.to_frame().T 
                    segment_inq_arr_for_bt_df = segment_inq_arr_for_bt_df.reindex(columns=sorted_fiscal_quarters_cols, fill_value=0)
                    segment_summary_df = pd.concat([segment_summary_df, segment_inq_arr_for_bt_df])
                total_inq_for_segment_series = segment_summary_df.sum(axis=0) 
                segment_summary_df.loc['Total Inquarter'] = total_inq_for_segment_series
                if segment_name in total_arr_by_seg_fp_unstacked.index:
                    total_bookings_for_segment_series = total_arr_by_seg_fp_unstacked.loc[segment_name].reindex(
                        index=sorted_fiscal_quarters_cols, fill_value=0 
                    )
                    segment_summary_df.loc['Total bookings'] = total_bookings_for_segment_series
                else: 
                    segment_summary_df.loc['Total bookings'] = 0 
                if 'Total Inquarter' in segment_summary_df.index and 'Total bookings' in segment_summary_df.index:
                    percent_inq_for_segment_series = (segment_summary_df.loc['Total Inquarter'] / segment_summary_df.loc['Total bookings'].replace(0, np.nan) * 100).fillna(0)
                    segment_summary_df.loc['Percent inquarter'] = percent_inq_for_segment_series.apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "0.00%")
                else:
                    segment_summary_df.loc['Percent inquarter'] = "N/A"
                segment_summary_df.index = pd.MultiIndex.from_product([[segment_name], segment_summary_df.index], names=['Segment - historical', 'Bookings Type / Metric'])
                all_segments_summary_list.append(segment_summary_df)
            if all_segments_summary_list:
                summary_by_segment_bookings_type_final = pd.concat(all_segments_summary_list)

    if not summary_by_segment_bookings_type_final.empty:
        cols_to_keep_seg = [col for col in summary_by_segment_bookings_type_final.columns if isinstance(col, str) and col >= fy_start_str_display]
        summary_by_segment_bt_filtered = summary_by_segment_bookings_type_final[sorted(cols_to_keep_seg)]
        
        # Add comparison columns for each segment separately
        segments = summary_by_segment_bt_filtered.index.get_level_values(0).unique()
        segment_dataframes = []
        
        for segment in segments:
            segment_data = summary_by_segment_bt_filtered.loc[segment].copy()
            # Identify numeric rows (exclude Percent inquarter)
            numeric_rows = segment_data.index[segment_data.index != 'Percent inquarter']
            if len(numeric_rows) > 0:
                segment_numeric = segment_data.loc[numeric_rows].copy()
                segment_numeric = add_comparison_columns(segment_numeric)
                
                # Add back percent row if it exists
                if 'Percent inquarter' in segment_data.index:
                    percent_row = segment_data.loc[['Percent inquarter']].copy()
                    # Add empty comparison columns to percent row
                    for col in ['QoQ Change', 'vs Last 4Q Avg', 'YoY Change']:
                        percent_row[col] = 'N/A'
                    segment_data = pd.concat([segment_numeric, percent_row])
                else:
                    segment_data = segment_numeric
            
            # Restore the multi-index
            segment_data.index = pd.MultiIndex.from_product([[segment], segment_data.index], names=['Segment - historical', 'Bookings Type / Metric'])
            segment_dataframes.append(segment_data)
        
        if segment_dataframes:
            summary_by_segment_bt_filtered = pd.concat(segment_dataframes)
        
        print(f"\nDisplaying: Summary by Segment & Bookings Type (Filtered to display from {fy_start_str_display})")
        # 3. --- CHANGED --- Use display() here as well for the second table
        display(summary_by_segment_bt_filtered)
    else:
        print("Could not generate or display 'Summary by Segment & Bookings Type'.")
else:
    print("Main summary table generation skipped: `processed_df` is empty or critical columns for summary are missing.")

print("--- Finished Attempting to Generate Main Summary Tables ---")

In [ ]:
# ## Comprehensive Trend Analysis: Complete Data Readout (No Threshold Filtering)
# 
# This section performs a comprehensive trend analysis and captures ALL comparisons.
# It compares each quarter against three different benchmarks:
# 1.  **Quarter-over-Quarter (QoQ):** Performance vs. the immediately preceding quarter.
# 2.  **Year-over-Year (YoY):** Performance vs. the same quarter in the prior fiscal year.
# 3.  **vs. 4-Quarter Average:** Performance vs. the average of the four preceding quarters.
#
# Unlike the threshold-based version, this captures EVERY comparison for user self-service analysis.

def clean_group_label(group_label):
    """
    Clean up group labels by removing verbose prefixes.
    """
    # Remove common prefixes
    prefixes_to_remove = [
        "By Segment & Bookings Type: ",
        "By Bookings Type: ",
        "By Segment: ",
        "Overall Inquarter",
        "Overall Total"
    ]
    
    cleaned_label = group_label
    for prefix in prefixes_to_remove:
        if cleaned_label.startswith(prefix):
            cleaned_label = cleaned_label[len(prefix):]
            break
    
    # Handle the "Overall" cases
    if group_label == "Overall Inquarter":
        cleaned_label = "Overall Inquarter"
    elif group_label == "Overall Total":
        cleaned_label = "Overall Total"
    
    return cleaned_label

def record_all_changes(comparison_df, current_q, comparison_label, group_label="Overall"):
    """
    Records ALL metric changes without any threshold filtering.
    Returns a list of all comparisons for analysis.
    """
    all_changes_list = []
    if comparison_df.empty: 
        return all_changes_list
    
    METRICS_TO_ANALYZE = ['Total ARR', '# of Opps', '# of Won Opps', 'Won ARR', 'Avg Sales Price', 'Win Rate (Count)', 'Avg Sales Cycle']
    
    for metric_name in METRICS_TO_ANALYZE:
        if metric_name not in comparison_df.index: 
            continue
            
        row_data = comparison_df.loc[metric_name]
        current_val = row_data.get('Current Quarter', 0)
        comparison_val = row_data.get(comparison_label, 0)
        pct_change = row_data.get('Percent Change', 0)
        abs_change = row_data.get('Absolute Change', 0)
        
        # Convert to numeric, handling any non-numeric values
        try:
            current_val = float(current_val) if pd.notnull(current_val) else 0
            comparison_val = float(comparison_val) if pd.notnull(comparison_val) else 0
            pct_change = float(pct_change) if pd.notnull(pct_change) else 0
            abs_change = float(abs_change) if pd.notnull(abs_change) else 0
        except (ValueError, TypeError):
            continue
            
        all_changes_list.append({
            'Quarter': current_q,
            'Comparison_Type': comparison_label,
            'Group': clean_group_label(group_label),
            'Metric': metric_name,
            'Current_Value': current_val,
            'Comparison_Value': comparison_val,
            'Absolute_Change': abs_change,
            'Percent_Change': pct_change,
            'Direction': 'Increase' if abs_change > 0 else ('Decrease' if abs_change < 0 else 'No Change')
        })
    
    return all_changes_list

# Initialize the complete trends list
all_trends_complete = []

processed_df_for_trends = pd.DataFrame()
inq_df_for_trends = pd.DataFrame()
fy_start_str_trends = f"FY{START_FISCAL_YEAR_FOR_ANALYSIS}Q1"

if not processed_df.empty and 'Fiscal Period - Corrected' in processed_df.columns:
    processed_df_for_trends = processed_df[processed_df['Fiscal Period - Corrected'] >= fy_start_str_trends].copy()
    if not inq_df.empty:
        inq_df_for_trends = inq_df[inq_df['Fiscal Period - Corrected'] >= fy_start_str_trends].copy()

if not processed_df_for_trends.empty and OPP_ID_COL_NAME and OPP_ID_COL_NAME in processed_df_for_trends.columns:
    
    grouping_definitions = {
        "Overall Inquarter": {'df': inq_df_for_trends, 'cols': None},
        "Overall Total": {'df': processed_df_for_trends, 'cols': None},
        "By Bookings Type": {'df': processed_df_for_trends, 'cols': 'Bookings Type'},
        "By Segment": {'df': processed_df_for_trends, 'cols': 'Segment - historical'},
        "By Segment & Bookings Type": {'df': processed_df_for_trends, 'cols': ['Segment - historical', 'Bookings Type']}
    }
    
    for group_name, G in grouping_definitions.items():
        cols_to_check = G['cols']
        if isinstance(cols_to_check, str): cols_to_check = [cols_to_check]
        
        if G['df'].empty or (cols_to_check and any(c not in G['df'].columns for c in cols_to_check)):
            print(f"Skipping grouping '{group_name}' due to missing data or columns.")
            continue
        
        print(f"Processing trends for: {group_name}")
        
        all_q_metrics = calculate_quarterly_metrics(G['df'], OPP_ID_COL_NAME, group_by_cols=G['cols'])
        if all_q_metrics.empty:
            print(f"Could not calculate quarterly metrics for {group_name}.")
            continue
            
        quarters_in_data = sorted([q for q in all_q_metrics.index.get_level_values('Fiscal Period - Corrected').unique() if q is not None])
        
        for i, current_q in enumerate(quarters_in_data):
            current_q_groups = all_q_metrics[all_q_metrics.index.get_level_values('Fiscal Period - Corrected') == current_q]
            if current_q_groups.empty: continue
            
            # --- Comparison 1: QoQ ---
            if i > 0:
                prev_q = quarters_in_data[i-1]
                comparison_label = f"QoQ vs {prev_q}"
                if G['cols']: 
                    for group_tuple in current_q_groups.index.droplevel('Fiscal Period - Corrected'):
                        group_as_tuple = group_tuple if isinstance(group_tuple, tuple) else (group_tuple,)
                        idx_key = group_as_tuple + (prev_q,)
                        if idx_key in all_q_metrics.index:
                             comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[idx_key], comparison_label, group_tuple=group_tuple)
                             group_label_str = " - ".join(map(str, group_as_tuple))
                             all_trends_complete.extend(record_all_changes(comparison_table, current_q, comparison_label, group_label_str))
                else: # Not grouped
                    if prev_q in all_q_metrics.index:
                        comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[prev_q], comparison_label)
                        all_trends_complete.extend(record_all_changes(comparison_table, current_q, comparison_label, group_name))

            # --- Comparison 2: YoY ---
            year, q_num = int(current_q[2:6]), int(current_q[-1])
            prev_year_q = f"FY{year-1}Q{q_num}"
            if prev_year_q in quarters_in_data:
                comparison_label = f"YoY vs {prev_year_q}"
                if G['cols']:
                    for group_tuple in current_q_groups.index.droplevel('Fiscal Period - Corrected'):
                        group_as_tuple = group_tuple if isinstance(group_tuple, tuple) else (group_tuple,)
                        idx_key = group_as_tuple + (prev_year_q,)
                        if idx_key in all_q_metrics.index:
                            comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[idx_key], comparison_label, group_tuple=group_tuple)
                            group_label_str = " - ".join(map(str, group_as_tuple))
                            all_trends_complete.extend(record_all_changes(comparison_table, current_q, comparison_label, group_label_str))
                else: # Not grouped
                    if prev_year_q in all_q_metrics.index:
                        comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[prev_year_q], comparison_label)
                        all_trends_complete.extend(record_all_changes(comparison_table, current_q, comparison_label, group_name))

            # --- Comparison 3: vs. Prior 4-Quarter Average ---
            if i >= 4:
                prior_4_quarters = quarters_in_data[i-4:i]
                comparison_label = f"vs 4Q Avg ({prior_4_quarters[0]}-{prior_4_quarters[-1]})"
                if G['cols']:
                    for group_tuple in current_q_groups.index.droplevel('Fiscal Period - Corrected'):
                        try:
                            # Select all data for the specific group, then filter for the prior 4 quarters
                            data_for_group = all_q_metrics.loc[group_tuple]
                            prior_4q_group_data = data_for_group.loc[data_for_group.index.isin(prior_4_quarters)]
                            
                            # Only proceed if we have data for all 4 prior quarters for this specific group
                            if len(prior_4q_group_data) == 4:
                                prior_4q_avg_group = prior_4q_group_data.mean()
                                comparison_table = build_comparison_view(all_q_metrics, current_q, prior_4q_avg_group, comparison_label, group_tuple=group_tuple)
                                group_as_tuple = group_tuple if isinstance(group_tuple, tuple) else (group_tuple,)
                                group_label_str = " - ".join(map(str, group_as_tuple))
                                all_trends_complete.extend(record_all_changes(comparison_table, current_q, comparison_label, group_label_str))
                        except (KeyError, IndexError):
                            continue
                else: # Not grouped
                    prior_4q_avg = all_q_metrics.loc[prior_4_quarters].mean()
                    comparison_table = build_comparison_view(all_q_metrics, current_q, prior_4q_avg, comparison_label)
                    all_trends_complete.extend(record_all_changes(comparison_table, current_q, comparison_label, group_name))

    # Create the complete trends DataFrame
    if all_trends_complete:
        print(f"\n{'='*25} COMPLETE TRENDS ANALYSIS READOUT {'='*25}")
        complete_trends_df = pd.DataFrame(all_trends_complete)
        
        # Reorder columns for better readability
        column_order = ['Quarter', 'Comparison_Type', 'Group', 'Metric', 'Current_Value', 'Comparison_Value', 
                       'Absolute_Change', 'Percent_Change', 'Direction']
        complete_trends_df = complete_trends_df[column_order]
        
        # Sort for logical viewing
        complete_trends_df = complete_trends_df.sort_values(by=['Quarter', 'Group', 'Comparison_Type', 'Metric'])
        
        # Display summary stats
        print(f"Total trend comparisons captured: {len(complete_trends_df)}")
        print(f"Quarters analyzed: {complete_trends_df['Quarter'].nunique()}")
        print(f"Groups analyzed: {complete_trends_df['Group'].nunique()}")
        print(f"Metrics tracked: {complete_trends_df['Metric'].nunique()}")
        
        # Show a sample of the data
        print(f"\nSample of complete trends data (first 10 rows):")
        display(complete_trends_df.head(10))
        
        # Save to CSV for user analysis
        complete_trends_df.to_csv('complete_trends_analysis.csv', index=False)
        print(f"\nComplete trends analysis saved to 'complete_trends_analysis.csv'")
        print("Users can now filter and analyze this data based on their own criteria.")
        
    else:
        print("\n\nNo trend data could be generated.")
else:
    print("Complete trends analysis skipped: Processed DataFrame is empty or critical columns are missing.")

In [ ]:
# ## Win Rate Analysis - 6 Row Structure (Using Module)
#
# This section performs 6-row win rate analysis using the modular approach.

from metrics_calculator import calculate_win_rate_analysis

print("🎯 WIN RATE 6-ROW ANALYSIS")
print("=" * 40)

# Check if processed_df exists and has required data
if 'processed_df' in locals() and not processed_df.empty and 'OPP_ID_COL_NAME' in locals():
    # Calculate win rate analysis
    win_rate_result = calculate_win_rate_analysis(
        df=processed_df,
        opp_id_col=OPP_ID_COL_NAME,
        stage_col='Stage',
        segment_col='Segment - historical',
        quarter_col='Fiscal Period - Corrected'
    )
    
    if not win_rate_result.empty:
        print("\n✅ Win Rate 6-row analysis completed successfully!")
        print("\n📊 Win Rate Analysis Results:")
        for index, row in win_rate_result.iterrows():
            print(f"{row['Metric']:<25}: {row['Win Rate (%)']:>8.1f}%")
        
        print("\n📋 Full Results DataFrame:")
        display(win_rate_result)
    else:
        print("❌ Could not generate win rate analysis results")
        
else:
    print("❌ Required data not found. Please ensure:")
    print("   1. processed_df exists (run Data Preprocessing cell)")
    print("   2. OPP_ID_COL_NAME is defined")
    print("   3. Required columns exist in processed_df")

In [ ]:
# ## ASP (Average Sales Price) Analysis - 6 Row Structure (Using Module)
#
# This section performs 6-row Average Sales Price analysis using the modular approach.

from metrics_calculator import calculate_asp_analysis

print("🎯 AVERAGE SALES PRICE 6-ROW ANALYSIS")
print("=" * 45)

# Check if processed_df exists and has required data
if 'processed_df' in locals() and not processed_df.empty and 'OPP_ID_COL_NAME' in locals():
    # Calculate ASP analysis
    asp_result = calculate_asp_analysis(
        df=processed_df,
        arr_col='ARR Change',
        opp_id_col=OPP_ID_COL_NAME,
        stage_col='Stage',
        segment_col='Segment - historical',
        quarter_col='Fiscal Period - Corrected'
    )
    
    if not asp_result.empty:
        print("\n✅ ASP 6-row analysis completed successfully!")
        print("\n📊 Average Sales Price Analysis Results:")
        for index, row in asp_result.iterrows():
            print(f"{row['Metric']:<25}: ${row['Avg Sales Price ($)']:>12,.0f}")
        
        print("\n📋 Full Results DataFrame:")
        display(asp_result)
    else:
        print("❌ Could not generate ASP analysis results")
        
else:
    print("❌ Required data not found. Please ensure:")
    print("   1. processed_df exists (run Data Preprocessing cell)")
    print("   2. OPP_ID_COL_NAME is defined")
    print("   3. Required columns exist in processed_df")

In [ ]:
# ## ASC (Average Sales Cycle) Analysis - 6 Row Structure (Using Module)
#
# This section performs 6-row Average Sales Cycle analysis using the modular approach.

from metrics_calculator import calculate_asc_analysis

print("🎯 AVERAGE SALES CYCLE 6-ROW ANALYSIS")
print("=" * 45)

# Check if processed_df exists and has required data
if 'processed_df' in locals() and not processed_df.empty and 'OPP_ID_COL_NAME' in locals():
    # Calculate ASC analysis - returns two DataFrames (simple and multi-index versions)
    asc_simple, asc_multi_index = calculate_asc_analysis(
        df=processed_df,
        opp_id_col=OPP_ID_COL_NAME,
        stage_col='Stage',
        segment_col='Bookings Type',
        quarter_col='Fiscal Period - Corrected',
        created_col='Created Date',
        close_col='Close Date'
    )
    
    if not asc_simple.empty:
        print("\n✅ ASC 6-row analysis completed successfully!")
        
        print("\n📊 Simple ASC Analysis (Bookings Type View):")
        display(asc_simple)
        
        if not asc_multi_index.empty:
            print("\n📊 Detailed ASC Analysis (Segment & Bookings Type View):")
            display(asc_multi_index)
        else:
            print("\n⚠️ Multi-index ASC analysis empty - no segment breakdown available")
            
    else:
        print("❌ Could not generate ASC analysis results")
        
else:
    print("❌ Required data not found. Please ensure:")
    print("   1. processed_df exists (run Data Preprocessing cell)")
    print("   2. OPP_ID_COL_NAME is defined")
    print("   3. Required columns exist in processed_df")

In [ ]:
# ## Comprehensive Trend Analysis: Comparing QoQ, YoY, and vs. 4-Quarter Average
# 
# This section performs a comprehensive trend analysis. It compares each quarter against three different benchmarks:
# 1.  **Quarter-over-Quarter (QoQ):** Performance vs. the immediately preceding quarter.
# 2.  **Year-over-Year (YoY):** Performance vs. the same quarter in the prior fiscal year.
# 3.  **vs. 4-Quarter Average:** Performance vs. the average of the four preceding quarters.
#
# This is done for multiple data slices (Overall, By Segment, etc.) and all significant findings are collected into a single summary table.

all_significant_changes_summary = []

processed_df_for_trends = pd.DataFrame()
inq_df_for_trends = pd.DataFrame()
fy_start_str_trends = f"FY{START_FISCAL_YEAR_FOR_ANALYSIS}Q1"

if not processed_df.empty and 'Fiscal Period - Corrected' in processed_df.columns:
    processed_df_for_trends = processed_df[processed_df['Fiscal Period - Corrected'] >= fy_start_str_trends].copy()
    if not inq_df.empty:
        inq_df_for_trends = inq_df[inq_df['Fiscal Period - Corrected'] >= fy_start_str_trends].copy()

if not processed_df_for_trends.empty and OPP_ID_COL_NAME and OPP_ID_COL_NAME in processed_df_for_trends.columns:
    
    grouping_definitions = {
        "Overall Inquarter": {'df': inq_df_for_trends, 'cols': None},
        "Overall Total": {'df': processed_df_for_trends, 'cols': None},
        "By Bookings Type": {'df': processed_df_for_trends, 'cols': 'Bookings Type'},
        "By Segment": {'df': processed_df_for_trends, 'cols': 'Segment - historical'},
        "By Segment & Bookings Type": {'df': processed_df_for_trends, 'cols': ['Segment - historical', 'Bookings Type']}
    }
    
    for group_name, G in grouping_definitions.items():
        cols_to_check = G['cols']
        if isinstance(cols_to_check, str): cols_to_check = [cols_to_check]
        
        if G['df'].empty or (cols_to_check and any(c not in G['df'].columns for c in cols_to_check)):
            print(f"Skipping grouping '{group_name}' due to missing data or columns.")
            continue
        
        print(f"\n\n{'='*25} Analyzing Trends for: {group_name} {'='*25}")
        
        all_q_metrics = calculate_quarterly_metrics(G['df'], OPP_ID_COL_NAME, group_by_cols=G['cols'])
        if all_q_metrics.empty:
            print(f"Could not calculate quarterly metrics for {group_name}.")
            continue
            
        quarters_in_data = sorted([q for q in all_q_metrics.index.get_level_values('Fiscal Period - Corrected').unique() if q is not None])
        
        for i, current_q in enumerate(quarters_in_data):
            current_q_groups = all_q_metrics[all_q_metrics.index.get_level_values('Fiscal Period - Corrected') == current_q]
            if current_q_groups.empty: continue
            
            # --- Comparison 1: QoQ ---
            if i > 0:
                prev_q = quarters_in_data[i-1]
                comparison_label = f"vs Prev Q ({prev_q})"
                if G['cols']: 
                    for group_tuple in current_q_groups.index.droplevel('Fiscal Period - Corrected'):
                        group_as_tuple = group_tuple if isinstance(group_tuple, tuple) else (group_tuple,)
                        idx_key = group_as_tuple + (prev_q,)
                        if idx_key in all_q_metrics.index:
                             comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[idx_key], comparison_label, group_tuple=group_tuple)
                             group_label_str = " - ".join(map(str, group_as_tuple))
                             all_significant_changes_summary.extend(identify_and_report_significant_changes(comparison_table, current_q, comparison_label, f"{group_name}: {group_label_str}"))
                else: # Not grouped
                    if prev_q in all_q_metrics.index:
                        comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[prev_q], comparison_label)
                        all_significant_changes_summary.extend(identify_and_report_significant_changes(comparison_table, current_q, comparison_label, group_name))

            # --- Comparison 2: YoY ---
            year, q_num = int(current_q[2:6]), int(current_q[-1])
            prev_year_q = f"FY{year-1}Q{q_num}"
            if prev_year_q in quarters_in_data:
                comparison_label = f"vs Prev Year ({prev_year_q})"
                if G['cols']:
                    for group_tuple in current_q_groups.index.droplevel('Fiscal Period - Corrected'):
                        group_as_tuple = group_tuple if isinstance(group_tuple, tuple) else (group_tuple,)
                        idx_key = group_as_tuple + (prev_year_q,)
                        if idx_key in all_q_metrics.index:
                            comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[idx_key], comparison_label, group_tuple=group_tuple)
                            group_label_str = " - ".join(map(str, group_as_tuple))
                            all_significant_changes_summary.extend(identify_and_report_significant_changes(comparison_table, current_q, comparison_label, f"{group_name}: {group_label_str}"))
                else: # Not grouped
                    if prev_year_q in all_q_metrics.index:
                        comparison_table = build_comparison_view(all_q_metrics, current_q, all_q_metrics.loc[prev_year_q], comparison_label)
                        all_significant_changes_summary.extend(identify_and_report_significant_changes(comparison_table, current_q, comparison_label, group_name))

            # --- Comparison 3: vs. Prior 4-Quarter Average ---
            if i >= 4:
                prior_4_quarters = quarters_in_data[i-4:i]
                comparison_label = f"vs Avg of {prior_4_quarters[0]}-{prior_4_quarters[-1]}"
                if G['cols']:
                    for group_tuple in current_q_groups.index.droplevel('Fiscal Period - Corrected'):
                        try:
                            # Select all data for the specific group, then filter for the prior 4 quarters
                            data_for_group = all_q_metrics.loc[group_tuple]
                            prior_4q_group_data = data_for_group.loc[data_for_group.index.isin(prior_4_quarters)]
                            
                            # Only proceed if we have data for all 4 prior quarters for this specific group
                            if len(prior_4q_group_data) == 4:
                                prior_4q_avg_group = prior_4q_group_data.mean()
                                comparison_table = build_comparison_view(all_q_metrics, current_q, prior_4q_avg_group, comparison_label, group_tuple=group_tuple)
                                group_as_tuple = group_tuple if isinstance(group_tuple, tuple) else (group_tuple,)
                                group_label_str = " - ".join(map(str, group_as_tuple))
                                all_significant_changes_summary.extend(identify_and_report_significant_changes(comparison_table, current_q, comparison_label, f"{group_name}: {group_label_str}"))
                        except (KeyError, IndexError):
                            continue
                else: # Not grouped
                    prior_4q_avg = all_q_metrics.loc[prior_4_quarters].mean()
                    comparison_table = build_comparison_view(all_q_metrics, current_q, prior_4q_avg, comparison_label)
                    all_significant_changes_summary.extend(identify_and_report_significant_changes(comparison_table, current_q, comparison_label, group_name))

    if all_significant_changes_summary:
        print(f"\n\n{'='*25} SUMMARY OF ALL SIGNIFICANT TRENDS {'='*25}")
        significant_changes_summary_df = pd.DataFrame(all_significant_changes_summary)
        summary_cols_order = ['Quarter', 'Comparison', 'Group', 'Metric', 'Type', '% Change', 'Abs Change']
        print(significant_changes_summary_df[summary_cols_order].sort_values(by=['Quarter', 'Comparison', 'Group']))
    else:
        print("\n\nNo significant changes met the defined thresholds across all comparisons.")
else:
    print("Significant changes summary skipped: Processed DataFrame is empty or critical columns are missing.")

if 'significant_changes_summary_df' in locals() and not significant_changes_summary_df.empty:
    significant_changes_summary_df.to_csv('significant_trends_summary.csv', index=False)
    print("\nSuccessfully saved the significant trends summary to 'significant_trends_summary.csv'")

In [ ]:
# ## Prophet Forecasting for In-Quarter Bookings
#
# ### Data Preparation for Prophet Models

prophet_data_overall = pd.DataFrame()
prophet_data_by_segment = {}
prophet_data_by_booking_type = {}

inq_df_for_prophet = pd.DataFrame()
fy_start_str_prophet = f"FY{START_FISCAL_YEAR_FOR_ANALYSIS}Q1"
if not inq_df.empty and 'Fiscal Period - Corrected' in inq_df.columns:
    inq_df_for_prophet = inq_df[inq_df['Fiscal Period - Corrected'] >= fy_start_str_prophet].copy()
    print(f"Data for Prophet models filtered to start from {fy_start_str_prophet}. Shape: {inq_df_for_prophet.shape}")

# Use 'ARR Change' for the 'y' value
if not inq_df_for_prophet.empty and 'Close Date' in inq_df_for_prophet.columns and 'ARR Change' in inq_df_for_prophet.columns:
    # Overall data
    daily_inq_arr_overall_temp = inq_df_for_prophet.groupby(pd.Grouper(key='Close Date', freq='D'))['ARR Change'].sum().reset_index()
    daily_inq_arr_overall_temp.rename(columns={'Close Date': 'ds', 'ARR Change': 'y'}, inplace=True)
    prophet_data_overall = daily_inq_arr_overall_temp[daily_inq_arr_overall_temp['ds'].notna()].copy()
    if not prophet_data_overall.empty: 
        print(f"Overall daily in-quarter data prepared for Prophet. Shape: {prophet_data_overall.shape}")
    else: 
        print("No overall data for Prophet after preparation.")

    # Data by Segment - historical
    if 'Segment - historical' in inq_df_for_prophet.columns:
        for seg_name in inq_df_for_prophet['Segment - historical'].dropna().unique():
            segment_data_temp = inq_df_for_prophet[inq_df_for_prophet['Segment - historical'] == seg_name]
            daily_segment_arr_temp = segment_data_temp.groupby(pd.Grouper(key='Close Date', freq='D'))['ARR Change'].sum().reset_index()
            daily_segment_arr_temp.rename(columns={'Close Date': 'ds', 'ARR Change': 'y'}, inplace=True)
            prophet_data_by_segment[seg_name] = daily_segment_arr_temp[daily_segment_arr_temp['ds'].notna()].copy()
            if not prophet_data_by_segment[seg_name].empty: 
                print(f"Segment '{seg_name}' daily data prepared for Prophet. Shape: {prophet_data_by_segment[seg_name].shape}")

    # Data by Bookings Type
    if 'Bookings Type' in inq_df_for_prophet.columns:
        for bt_name in inq_df_for_prophet['Bookings Type'].dropna().unique():
            booking_type_data_temp = inq_df_for_prophet[inq_df_for_prophet['Bookings Type'] == bt_name]
            daily_bt_arr_temp = booking_type_data_temp.groupby(pd.Grouper(key='Close Date', freq='D'))['ARR Change'].sum().reset_index()
            daily_bt_arr_temp.rename(columns={'Close Date': 'ds', 'ARR Change': 'y'}, inplace=True)
            prophet_data_by_booking_type[bt_name] = daily_bt_arr_temp[daily_bt_arr_temp['ds'].notna()].copy()
            if not prophet_data_by_booking_type[bt_name].empty: 
                print(f"Booking Type '{bt_name}' daily data prepared for Prophet. Shape: {prophet_data_by_booking_type[bt_name].shape}")
else:
    print("Prophet data preparation skipped: `inq_df_for_prophet` is empty or essential columns are missing.")

In [ ]:
# ### Prophet Forecasting: Model Training & Prediction Function

def run_prophet_forecast(time_series_df, series_descriptive_name="Series", future_forecast_periods=92, 
                         seasonality_mode_prophet='additive', yearly_seasonality_prophet=True, 
                         weekly_seasonality_prophet=False, daily_seasonality_prophet=False):
    
    if time_series_df.empty or len(time_series_df) < 2: # Prophet needs at least 2 data points
        print(f"Prophet Warning: Not enough data for model for {series_descriptive_name}. Min 2 data points required, got {len(time_series_df)}.")
        return None, pd.DataFrame()

    print(f"\nRunning Prophet for: {series_descriptive_name}")
    
    prophet_model_instance = Prophet(
        yearly_seasonality=yearly_seasonality_prophet,
        weekly_seasonality=weekly_seasonality_prophet,
        daily_seasonality=daily_seasonality_prophet,
        seasonality_mode=seasonality_mode_prophet,
        changepoint_prior_scale=0.05 # Default, adjust if over/underfitting trend
    )
    
    # Optional: Add custom fiscal quarterly seasonality if yearly doesn't capture it well enough for your specific fiscal calendar
    # This assumes a standard calendar year division for quarters. Adjust 'period' if your fiscal quarters have very specific lengths.
    # fiscal_quarter_period_days = 365.25 / 4 
    # prophet_model_instance.add_seasonality(name='fiscal_quarterly', period=fiscal_quarter_period_days, fourier_order=5) # fourier_order controls complexity

    try:
        prophet_model_instance.fit(time_series_df)
    except Exception as e:
        print(f"ERROR fitting Prophet model for {series_descriptive_name}: {e}")
        print("Data sample (first 5 rows):")
        print(time_series_df.head())
        return None, pd.DataFrame()

    future_dates_df = prophet_model_instance.make_future_dataframe(periods=future_forecast_periods, freq='D')
    forecast_results_df = prophet_model_instance.predict(future_dates_df)

    print(f"Prophet forecast for {series_descriptive_name} completed.")

    # Plotting Forecast
    try:
        fig_forecast = prophet_model_instance.plot(forecast_results_df)
        plt.title(f'Forecast for {series_descriptive_name}', fontsize=16)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Forecasted Value (yhat)', fontsize=12)
        plt.show()
    except Exception as e: print(f"Error plotting forecast for {series_descriptive_name}: {e}")

    # Plotting Components
    try:
        fig_components = prophet_model_instance.plot_components(forecast_results_df)
        fig_components.suptitle(f'Forecast Components for {series_descriptive_name}', fontsize=16, y=1.03) # Adjust y for title position
        plt.show()
    except Exception as e: print(f"Error plotting components for {series_descriptive_name}: {e}")

    # Merge actuals 'y' back to forecast for easier comparison if needed
    forecast_results_df_with_actuals = pd.merge(forecast_results_df, time_series_df[['ds', 'y']], on='ds', how='left')

    return prophet_model_instance, forecast_results_df_with_actuals

print("Cell 10: `run_prophet_forecast` function - Defined")

In [ ]:
# ### Prophet Forecasting: Execution & Storing Models/Forecasts

DAYS_TO_FORECAST_PROPHET = 92 # Typical quarter length

all_prophet_models = {}
all_prophet_forecasts = {}

# 1. Overall In-Quarter Bookings Forecast
if not prophet_data_overall.empty:
    model_overall_instance, forecast_overall_df = run_prophet_forecast(
        prophet_data_overall,
        series_descriptive_name="Overall In-Quarter Bookings",
        future_forecast_periods=DAYS_TO_FORECAST_PROPHET
    )
    if model_overall_instance: # Check if model training was successful
        all_prophet_models['overall'] = model_overall_instance
        all_prophet_forecasts['overall'] = forecast_overall_df
        print(f"--- Overall Forecast Details (tail) ---")
        print(forecast_overall_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'y']].tail())
else:
    print("Skipping Overall Prophet forecast as prepared data is empty.")

# 2. In-Quarter Bookings Forecast by Segment
if prophet_data_by_segment: # Check if the dictionary itself is not empty
    for segment_name_key, segment_timeseries_df in prophet_data_by_segment.items():
        if not segment_timeseries_df.empty:
            model_segment_instance, forecast_segment_df = run_prophet_forecast(
                segment_timeseries_df,
                series_descriptive_name=f"Segment: {segment_name_key} In-Quarter Bookings",
                future_forecast_periods=DAYS_TO_FORECAST_PROPHET
            )
            if model_segment_instance:
                all_prophet_models[f'segment_{segment_name_key}'] = model_segment_instance
                all_prophet_forecasts[f'segment_{segment_name_key}'] = forecast_segment_df
                print(f"--- Forecast Details for Segment: {segment_name_key} (tail) ---")
                print(forecast_segment_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'y']].tail())
        else:
            print(f"Skipping Prophet forecast for segment '{segment_name_key}' as its prepared data is empty.")
else:
    print("Skipping Segment-level Prophet forecasts as no segment data was prepared.")

# 3. In-Quarter Bookings Forecast by Bookings Type
if prophet_data_by_booking_type: # Check if the dictionary itself is not empty
    for bt_name_key, bt_timeseries_df in prophet_data_by_booking_type.items():
        if not bt_timeseries_df.empty:
            model_bt_instance, forecast_bt_df = run_prophet_forecast(
                bt_timeseries_df,
                series_descriptive_name=f"Bookings Type: {bt_name_key} In-Quarter Bookings",
                future_forecast_periods=DAYS_TO_FORECAST_PROPHET
            )
            if model_bt_instance:
                all_prophet_models[f'booking_type_{bt_name_key}'] = model_bt_instance
                all_prophet_forecasts[f'booking_type_{bt_name_key}'] = forecast_bt_df
                print(f"--- Forecast Details for Bookings Type: {bt_name_key} (tail) ---")
                print(forecast_bt_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'y']].tail())
        else:
            print(f"Skipping Prophet forecast for bookings type '{bt_name_key}' as its prepared data is empty.")
else:
    print("Skipping Booking Type-level Prophet forecasts as no booking type data was prepared.")

In [ ]:
# ### Prophet Forecasting: Pacing Table Generation & "Unseen" Revenue Analysis

# Determine the start date for "future" pacing tables
# This would typically be the day after the last known data point in the overall series
PACING_TABLE_START_DATE = None
if not prophet_data_overall.empty and 'ds' in prophet_data_overall.columns and not prophet_data_overall['ds'].empty:
    PACING_TABLE_START_DATE = prophet_data_overall['ds'].max() + pd.Timedelta(days=1)
    print(f"Pacing tables will generate forecasts starting from (day after last actual): {PACING_TABLE_START_DATE.strftime('%Y-%m-%d')}")
else:
    print("Warning: Cannot determine PACING_TABLE_START_DATE from prophet_data_overall. Pacing tables might be impacted or skipped.")

def create_pacing_table_from_forecast(prophet_forecast_df, descriptive_name_series, future_pacing_start_date):
    if prophet_forecast_df.empty:
        print(f"Cannot create pacing table for {descriptive_name_series}, input forecast_df is empty.")
        return pd.DataFrame()
    if future_pacing_start_date is None:
        print(f"Cannot create pacing table for {descriptive_name_series}, future_pacing_start_date is None (likely no historical data to base it on).")
        return pd.DataFrame()

    # Filter for future dates based on the start date
    pacing_df_filtered = prophet_forecast_df[prophet_forecast_df['ds'] >= pd.to_datetime(future_pacing_start_date)].copy()
    
    # Ensure required columns from Prophet output are present
    required_prophet_cols = ['ds', 'yhat', 'yhat_lower', 'yhat_upper']
    if not all(col in pacing_df_filtered.columns for col in required_prophet_cols):
        print(f"Error: Pacing table for {descriptive_name_series} - forecast DataFrame is missing one or more required Prophet columns: {required_prophet_cols}")
        print("Available columns in filtered forecast:", pacing_df_filtered.columns.tolist())
        return pd.DataFrame()
        
    pacing_df_selected = pacing_df_filtered[required_prophet_cols].copy() # Select only needed columns
    
    # Ensure yhat (forecasted value) and its bounds are not negative (e.g. ARR cannot be negative)
    pacing_df_selected['yhat'] = np.maximum(0, pacing_df_selected['yhat'])
    pacing_df_selected['yhat_lower'] = np.maximum(0, pacing_df_selected['yhat_lower'])
    pacing_df_selected['yhat_upper'] = np.maximum(0, pacing_df_selected['yhat_upper'])
    
    # Calculate cumulative forecast using the (potentially floored) yhat
    pacing_df_selected['Cumulative Forecast (yhat)'] = pacing_df_selected['yhat'].cumsum()
    
    # Rename columns for clarity in the final pacing table
    pacing_df_selected.rename(columns={
        'ds': 'Date', 
        'yhat': 'Forecasted Daily Value', # More generic than ARR if used for other metrics
        'yhat_lower': 'Lower Bound Daily Value',
        'yhat_upper': 'Upper Bound Daily Value'
    }, inplace=True)
    
    print(f"\n--- Pacing Table for {descriptive_name_series} (Next {DAYS_TO_FORECAST_PROPHET} Days) ---")
    if not pacing_df_selected.empty:
        print(pacing_df_selected.head())
        total_forecasted_for_period = pacing_df_selected['Forecasted Daily Value'].sum()
        print(f"Total Forecasted Value for {descriptive_name_series} over next {DAYS_TO_FORECAST_PROPHET} days: {total_forecasted_for_period:,.2f}")
    else:
        print("Pacing table is empty (no future dates or other issue).")

    
    # Plot cumulative forecast for the pacing period
    if not pacing_df_selected.empty:
        plt.figure(figsize=(12, 6))
        plt.plot(pacing_df_selected['Date'], pacing_df_selected['Cumulative Forecast (yhat)'], 
                 label='Cumulative Forecast (yhat)', marker='.', linestyle='-')
        
        # Calculate cumulative sums for lower and upper bounds for the plot
        cumulative_lower_plot = pacing_df_selected['Lower Bound Daily Value'].cumsum()
        cumulative_upper_plot = pacing_df_selected['Upper Bound Daily Value'].cumsum()
        
        plt.fill_between(pacing_df_selected['Date'], 
                         cumulative_lower_plot, 
                         cumulative_upper_plot, 
                         alpha=0.2, label='Cumulative Confidence Interval')
        
        plt.title(f'Cumulative Forecasted Value - {descriptive_name_series}', fontsize=14)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Cumulative Value', fontsize=12)
        plt.legend()
        plt.grid(True, which='both', linestyle=':')
        plt.tight_layout()
        plt.show()
    
    return pacing_df_selected

# Generate Pacing Tables
if PACING_TABLE_START_DATE:
    if 'overall' in all_prophet_forecasts and not all_prophet_forecasts['overall'].empty:
        pacing_table_overall = create_pacing_table_from_forecast(
            all_prophet_forecasts['overall'], 
            "Overall In-Quarter Bookings", 
            PACING_TABLE_START_DATE
        )

    for forecast_key, forecast_data_df in all_prophet_forecasts.items():
        if forecast_key == 'overall': continue # Already handled

        if not forecast_data_df.empty:
            descriptive_name_for_pacing = forecast_key.replace('_', ' ').title() # e.g. "Segment Enterprise"
            create_pacing_table_from_forecast(
                forecast_data_df, 
                f"{descriptive_name_for_pacing} In-Quarter Bookings", 
                PACING_TABLE_START_DATE
            )
else:
    print("Pacing table generation skipped as PACING_TABLE_START_DATE could not be determined.")


In [ ]:
# **Discussion on "Unseen" Revenue:**
# The `Forecasted Daily Value` (yhat) in the pacing tables represents Prophet's prediction for that day's in-quarter bookings. This prediction is based on historical patterns and inherently includes bookings expected from opportunities that are not yet created or visible in the current pipeline (i.e., "unseen" at the start of the forecast period but are expected to be created and closed within the quarter based on learned trends). The cumulative sum shows the total expected in-quarter bookings for the forecast horizon.

# ### Prophet Forecasting: Seasonality Insights & Historical Quarterly Pacing Comparison

def plot_historical_quarterly_pacing(daily_actuals_df_prophet_format, series_descriptive_name="Series"):
    if daily_actuals_df_prophet_format.empty or 'ds' not in daily_actuals_df_prophet_format.columns or 'y' not in daily_actuals_df_prophet_format.columns:
        print(f"Cannot plot historical pacing for {series_descriptive_name}, input DataFrame is empty or missing 'ds'/'y'.")
        return

    plot_df_historical = daily_actuals_df_prophet_format.copy()
    
    # Apply get_fiscal_quarter_info and expand results into new columns
    # We need: quarter_start_date (idx 0), quarter_label_str (idx 1)
    fiscal_info_tuples = plot_df_historical['ds'].apply(lambda x: get_fiscal_quarter_info(x))
    plot_df_historical['Quarter Start Date'] = fiscal_info_tuples.apply(lambda x: x[0])
    plot_df_historical['Fiscal Quarter Label'] = fiscal_info_tuples.apply(lambda x: x[1])

    # Drop rows where fiscal information couldn't be derived (e.g., NaT dates)
    plot_df_historical.dropna(subset=['Fiscal Quarter Label', 'Quarter Start Date'], inplace=True)
    if plot_df_historical.empty: 
        print(f"No valid fiscal quarter data for {series_descriptive_name} after processing for historical pacing plot.")
        return

    # Calculate Day Number within the Fiscal Quarter
    plot_df_historical['Day of Quarter'] = (plot_df_historical['ds'] - plot_df_historical['Quarter Start Date']).dt.days + 1
    
    # Extract Quarter Type (Q1, Q2, Q3, Q4) for hue
    plot_df_historical['Quarter Type'] = plot_df_historical['Fiscal Quarter Label'].str[-2:] # Takes 'Q1', 'Q2' etc.

    plt.figure(figsize=(14, 7))
    sns.lineplot(data=plot_df_historical, x='Day of Quarter', y='y', hue='Quarter Type', errorbar=('ci', 95), legend='full') # ci for confidence interval
    plt.title(f'Historical Average Daily Value Pacing by Quarter Type - {series_descriptive_name}', fontsize=16)
    plt.xlabel('Day of Fiscal Quarter', fontsize=12)
    plt.ylabel('Average Daily Value (y)', fontsize=12)
    plt.legend(title='Quarter Type', bbox_to_anchor=(1.05, 1), loc='upper left') # Move legend outside
    plt.grid(True, which='both', linestyle=':', linewidth=0.7)
    
    max_day_observed = plot_df_historical['Day of Quarter'].max() if not plot_df_historical.empty else 95
    plt.xlim(0, max_day_observed + 1 if pd.notnull(max_day_observed) else 95) # Adjust x-limit
    plt.tight_layout()
    plt.show()

# Plot historical quarterly pacing for overall data
if not prophet_data_overall.empty:
    plot_historical_quarterly_pacing(prophet_data_overall, "Overall In-Quarter Bookings (Historical Data)")
else: 
    print("Skipping historical quarterly pacing plot for Overall data as prepared data is empty.")

# Plot for Segments
if prophet_data_by_segment:
    for segment_name, segment_df_for_prophet in prophet_data_by_segment.items():
        if not segment_df_for_prophet.empty:
            plot_historical_quarterly_pacing(segment_df_for_prophet, f"Segment: {segment_name} (Historical Data)")
        else:
            print(f"Skipping historical quarterly pacing for segment '{segment_name}' as its prepared data is empty.")

# Plot for Bookings Types
if prophet_data_by_booking_type:
    for bt_name, bt_df_for_prophet in prophet_data_by_booking_type.items():
        if not bt_df_for_prophet.empty:
            plot_historical_quarterly_pacing(bt_df_for_prophet, f"Bookings Type: {bt_name} (Historical Data)")
        else:
            print(f"Skipping historical quarterly pacing for bookings type '{bt_name}' as its prepared data is empty.")

print("\nCell 13: Prophet Seasonality & Historical Pacing Plots - Executed (if data was available)")

In [ ]:
# ## QUICK TEST: Verify Modular Functions Work
#
# This cell tests if the .py modules can be imported and work correctly

print("🔍 TESTING MODULAR FUNCTIONS")
print("=" * 40)

try:
    from date_utils import enhance_dataframe_dates
    print("✅ date_utils.py imported successfully")
except Exception as e:
    print(f"❌ Error importing date_utils: {e}")

try:
    from metrics_calculator import calculate_quarterly_metrics
    print("✅ metrics_calculator.py imported successfully")
except Exception as e:
    print(f"❌ Error importing metrics_calculator: {e}")

try:
    from trend_analyzer import perform_trend_analysis
    print("✅ trend_analyzer.py imported successfully")
except Exception as e:
    print(f"❌ Error importing trend_analyzer: {e}")

try:
    from pipegen_analyzer import process_arr_change_history
    print("✅ pipegen_analyzer.py imported successfully")
except Exception as e:
    print(f"❌ Error importing pipegen_analyzer: {e}")

# Test with sample data
try:
    import pandas as pd
    sample_df = pd.DataFrame({
        'Created Date': ['2024-01-15', '2024-02-20'],
        'Close Date': ['2024-01-20', '2024-02-25']
    })
    enhanced_df = enhance_dataframe_dates(sample_df)
    print(f"✅ Date enhancement test successful - added {len(enhanced_df.columns) - 2} new columns")
except Exception as e:
    print(f"❌ Date enhancement test failed: {e}")

print("\n🎯 MODULE TEST COMPLETE")
print("If all tests passed, the modules work correctly.")
print("The issue is likely missing variables from previous cells.")